# Pair Trading with Kalman Filter - Dynamic Hedge Ratio Analysis

## Objective
This exercise aims to use a Kalman filter to estimate the **dynamic hedge ratio** between pairs of related stocks for potential **pair trading**. Your goal is to identify which pair of stocks has the highest probability of yielding profitable trading signals.


In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from datetime import date
import warnings


warnings.filterwarnings('ignore')


### Task 1 - Data Preprocessing
1. Load and verify the quality of the data.
2. Aggregate the data to x-minute interval

In [ ]:
def get_data(): 
    A = pd.read_csv(r"data\A.csv")
    B = pd.read_csv(r"data\B.csv")
    C = pd.read_csv(r"data\C.csv")
    return A, B,C 

def data_analysis(df, name = "X"):
    # understanding data 
    print("Data Description:", "\n")
    print(df.describe())

    require = ["exg_time","trade_price","trade_qty"]
    missing = [c for c in require if c not in df.columns]
    if missing:
        return {"name": name, "error": f"missing columns: {missing}"}

    d = df.copy()
    # time stamp analysis 
    d["exg_time"] = pd.to_datetime(d["exg_time"], utc=True, errors="coerce")
    d = d.sort_values("exg_time")

    out = {}
    out["name"] = name
    out["rows"] = len(d)

    # nan values, missing values 
    out["null_exg_time"] = int(d["exg_time"].isna().sum())
    out["null_price"]    = int(d["trade_price"].isna().sum())
    out["null_qty"]      = int(d["trade_qty"].isna().sum())
    out["nonpos_price"]  = int((d["trade_price"] <= 0).sum())
    out["nonpos_qty"]    = int((d["trade_qty"] <= 0).sum())

    # time range
    out["start"] = str(d["exg_time"].min())
    out["end"]   = str(d["exg_time"].max())

    # check exact duplicated rows 
    keys = ["exg_time", "trade_price", "trade_qty"]

    dup_mask = df.duplicated(subset=keys, keep="first")   # marks ONLY the later repeats
    dup_count = int(dup_mask.sum())
    out["duplicated rows"] = dup_count 

    return out

def data_cleaning(df):
    TICK_KEYS = ["exg_time", "trade_price", "trade_qty"]
    d = df.copy()
    d["exg_time"] = pd.to_datetime(d["exg_time"], utc=True, errors="coerce")

    # drop null time
    d = d.dropna(subset=["exg_time"])

    # drop negative price or negative qty
    d = d[(d["trade_price"] > 0) & (d["trade_qty"] > 0)]

    # drop exact duplicates (keep first only)
    d = d.drop_duplicates(subset=TICK_KEYS, keep="first")

    # sort
    d = d.sort_values("exg_time").reset_index(drop=True)

    return d 


def resample_data(df, x_min = 1):




In [15]:
A, B, C = get_data()

print(data_analysis(A, name = "A"))
print(data_analysis(B, name = "B"))
print(data_analysis(C, name = "C"))

Data Description: 

         trade_price      trade_qty
count  460248.000000  460248.000000
mean    33513.581460     402.947759
std      1378.703901     780.900078
min     31600.000000     100.000000
25%     32320.000000     100.000000
50%     33430.000000     200.000000
75%     34120.000000     400.000000
max     36600.000000   72500.000000
{'name': 'A', 'rows': 460248, 'null_exg_time': 6, 'null_price': 0, 'null_qty': 0, 'nonpos_price': 0, 'nonpos_qty': 0, 'start': '2024-01-02 01:00:00.150894+00:00', 'end': '2024-04-03 09:04:55.015295+00:00', 'duplicated rows': 134292}
Data Description: 

         trade_price      trade_qty
count  339588.000000  339588.000000
mean    28559.525366     358.073901
std       494.355500     706.291635
min     27620.000000     100.000000
25%     28200.000000     100.000000
50%     28370.000000     200.000000
75%     28910.000000     300.000000
max     29880.000000  119900.000000
{'name': 'B', 'rows': 339588, 'null_exg_time': 0, 'null_price': 0, 'null_qty': 

### Comments on Data 
* Data is somewhat clean, some null "exg_time" in A.
* No null values in price and qty for all A,B,C
* A LOT of duplicated rows (exact values for all 3 columns:["exg_time", "trade_price", "trade_qty"]) 
* handled by dropping rows with null "exg_time" value + dropping all duplicated rows, only keeping the first instance. 

### Task 2 - Implement Kalman Filter for Hedge Ratio Estimation
1. Write a function to apply the **Kalman filter** and estimate the hedge ratio between two stocks dynamically over time.
2. Calculate the hedge ratio for each of the pairs


### Task 3 - Calculate the Spread for Each Pair
1. Using the hedge ratio estimated in Task 2, compute a **spread** for each pair
2. **Check Mean Reversion**: Before applying the strategy, check if the spread between the pairs shows signs of mean reversion.

### Task 4 - Simulate Trading Strategy and Profitability
1. Define a simple **mean-reversion trading strategy**, assuming no trading cost.
2. Calculate the **cumulative profit and loss (P&L)** for each pair based on this strategy.
3. Record the cumulative P&L for each pair at the end of the period.

In [12]:
## trading class 
class TradingEngine:
    def __init__(self, initial_capital=100000.0, commission_bps=0.0001):
        self.cash = initial_capital
        self.initial_capital = initial_capital
        self.commission_bps = commission_bps
        self.positions = {} # {ticker: {'qty': 0, 'avg_price': 0.0}}
        self.history = []
        self.equity_curve = [] # Track daily portfolio value

    def get_portfolio_value(self, current_prices):
        """Returns Cash + Market Value of Positions"""
        market_value = 0.0
        for ticker, pos in self.positions.items():
            if ticker in current_prices:
                market_value += pos['qty'] * current_prices[ticker]
        return self.cash + market_value

    def execute_order(self, ticker, qty, price, date, action_label):
        """
        Handles Buy/Sell logic including Cash deduction.
        """
        if qty == 0: 
            return

        # Calculate Cost & Commission
        notional = abs(qty * price)
        comm = notional * self.commission_bps 
        
        self.cash -= (qty * price) 
        self.cash -= comm
        
        # Update Position 
        current_pos = self.positions.get(ticker, {'qty': 0, 'avg_price': 0.0})
        curr_qty = current_pos['qty']
        
        # Check direction
        is_same_direction = (curr_qty * qty) > 0 or curr_qty == 0
        
        if is_same_direction:
            # scale in 
            new_total_qty = curr_qty + qty
            old_notional = curr_qty * current_pos['avg_price']
            new_trade_notional = qty * price
        
            new_avg_price = price
            if new_total_qty != 0:
                new_avg_price = (old_notional + new_trade_notional) / new_total_qty
            
            self.positions[ticker] = {'qty': new_total_qty, 'avg_price': new_avg_price}
            
        else:
            # update quantity 
            new_total_qty = curr_qty + qty
            if new_total_qty == 0:
                if ticker in self.positions: del self.positions[ticker]
            else:
                is_flip = (curr_qty * new_total_qty) < 0
                new_avg_price = price if is_flip else current_pos['avg_price']
                self.positions[ticker] = {'qty': new_total_qty, 'avg_price': new_avg_price}

        # log trade
        self.history.append({
            'date': date, 'ticker': ticker, 'action': action_label,
            'qty': qty, 'price': price, 'comm': comm
        })
        
def portfolio_analytics(equity: pd.Series, periods_per_year=252):
    equity = equity.dropna()
    rets = equity.pct_change().dropna()

    # Sharpe (rf=0)
    sharpe = np.nan
    if rets.std() != 0:
        sharpe = (rets.mean() / rets.std()) * np.sqrt(periods_per_year)

    # Drawdown
    peak = equity.cummax()
    dd = equity / peak - 1.0
    max_dd = dd.min()

    # Max DD duration (in days)
    underwater = equity < peak
    dd_duration = (underwater.groupby((underwater != underwater.shift()).cumsum())
                            .cumcount() + 1)
    max_dd_duration = dd_duration[underwater].max() if underwater.any() else 0

    # CAGR
    years = (equity.index[-1] - equity.index[0]).days / 365.25
    if years > 0:
        cagr = (equity.iloc[-1] / equity.iloc[0]) ** (1/years) - 1 
    else:
        cagr = np.nan

    # Vol (annualized)
    ann_vol = rets.std() * np.sqrt(periods_per_year)

    return {
        "Final Equity": float(equity.iloc[-1]),
        "Total Return": float(equity.iloc[-1] / equity.iloc[0] - 1),
        "CAGR": float(cagr),
        "Ann Vol": float(ann_vol),
        "Sharpe (rf=0)": float(sharpe),
        "Max Drawdown": float(max_dd),
        "Max DD Duration (days)": int(max_dd_duration),
    }, dd, rets

### Task 5 - Conclusion
1. Identify which pair produced the **highest cumulative P&L**.
2. Summarize your results and provide an interpretation of which pair is the most likely to yield a profitable trading opportunity.